## Prediction with everything

In [1]:
import pandas as pd
import re

%matplotlib inline

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
csvs = pd.read_csv('../data/master.csv', index_col='SK_ID_CURR').drop(columns='Unnamed: 0') 
sql = pd.read_csv('../data/processed/full_sql.csv', index_col='SK_ID_CURR')
api = pd.read_csv('../data/api_data_clean.csv', index_col='sk_id_curr')

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22,23,24,32,33,34,44,46,59,64,116) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
csvs.head(1)

,TOTALAREA_MODE,FLOORSMIN_AVG,FLOORSMIN_MODE,ORGANIZATION_TYPE,NONLIVINGAREA_AVG,HOUSETYPE_MODE,LANDAREA_AVG,DAYS_ID_PUBLISH,ELEVATORS_AVG,COMMONAREA_MODE,...,EXT_SOURCE_2,FLAG_DOCUMENT_5,FLAG_EMP_PHONE,FLAG_WORK_PHONE,IS_KAEL,LIVINGAREA_AVG,NAME_FAMILY_STATUS,REGION_RATING_CLIENT,REG_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE_CAT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100004,?,?,?,Government,?,?,?,-2531,?,?,...,55.59%,0,1,1,NaN,-99999.0,Single / not married,I hope this data helps you.Sorry_for_not_not h...,0,Government


In [5]:
sql.head(1)

,TARGET,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,BASEMENTAREA_AVG,ENTRANCES_AVG,FLAG_DOCUMENT_20,FONDKAPREMONT_MODE,CNT_CHILDREN,FLAG_DOCUMENT_15,FLAG_EMAIL,OWN_CAR_AGE
SK_ID_CURR,,,,,,,,,,,
100004,NaN,0.0,-815.0,NaN,NaN,0,NaN,0,0,0,26.0


In [6]:
api.head(1)

,days_birth,flag_document_13,flag_document_9,name_education_type,name_income_type,obs_30_cnt_social_circle
sk_id_curr,,,,,,
100004,-19046,False,False,Secondary / secondary special,Working,0


In [7]:
temp_1 = pd.merge(sql, csvs, how='left', right_index=True, left_index=True)

In [8]:
master = pd.merge(temp_1, api, how='left', right_index=True, left_index=True)

In [9]:
master.head()

,TARGET,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,BASEMENTAREA_AVG,ENTRANCES_AVG,FLAG_DOCUMENT_20,FONDKAPREMONT_MODE,CNT_CHILDREN,FLAG_DOCUMENT_15,FLAG_EMAIL,...,NAME_FAMILY_STATUS,REGION_RATING_CLIENT,REG_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE_CAT,days_birth,flag_document_13,flag_document_9,name_education_type,name_income_type,obs_30_cnt_social_circle
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100004,NaN,0.0,-815.0,NaN,NaN,0,NaN,0,0,0,...,Single / not married,I hope this data helps you.Sorry_for_not_not h...,0,Government,-19046,False,False,Secondary / secondary special,Working,0
100006,0.0,0.0,-617.0,NaN,NaN,0,NaN,0,0,0,...,Civil marriage,2,0,Business Entity Type 3,-19005,False,False,Secondary / secondary special,Working,2
100012,NaN,0.0,-1673.0,NaN,NaN,0,NaN,0,0,0,...,Single / not married,2,0,Electricity,-14469,False,False,Secondary / secondary special,Working,2
100014,0.0,0.0,-844.0,NaN,NaN,0,NaN,1,0,0,...,Married,2,0,Medicine,-10197,False,False,Higher education,Working,0
100015,0.0,0.0,-2396.0,NaN,NaN,0,NaN,0,0,0,...,Married,2,0,XNA,-20417,False,False,Secondary / secondary special,Pensioner,0


## Create the training and test datasets

In [10]:
train = master[~master.TARGET.isnull()]

In [11]:
test = master.reindex(master.index.difference(train.index))

In [12]:
train.shape

(75000, 134)

In [13]:
test.shape

(25000, 134)

In [14]:
a = []
for col in master.columns:
    if master[col].isnull().sum() > 50000:
        a.append(col)

print(a)

['BASEMENTAREA_AVG', 'ENTRANCES_AVG', 'FONDKAPREMONT_MODE', 'OWN_CAR_AGE', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'LIVINGAPARTMENTS_MEDI', 'WALLSMATERIAL_MODE_x', 'NONLIVINGAPARTMENTS_AVG_x', 'WALLSMATERIAL_MODE_y', 'NONLIVINGAPARTMENTS_AVG_y', 'APARTMENTS_MODE_x', 'NONLIVINGAREA_MEDI_x', 'APARTMENTS_MODE_y', 'FLAG_DOCUMENT_6_y', 'FLAG_OWN_REALTY_y', 'NONLIVINGAREA_MEDI_y', 'FLAG_DOCUMENT_3_y', 'FLAG_DOCUMENT_4_y', 'FLAG_DOCUMENT_18_y', 'NAME_TYPE_SUITE_y', 'FLOORSMAX_MEDI_y', 'AMT_REQ_CREDIT_BUREAU_DAY_y', 'EXT_SOURCE_1', 'LIVINGAREA_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'EXT_SOURCE_1.1', 'LIVINGAREA_MEDI.1', 'FLOORSMIN_MEDI.1', 'LANDAREA_MEDI.1']


## Flags

In [15]:
flags = []
pattern = re.compile("FLAG|flag")
for col in master.columns:
    if pattern.search(col):
        flags.append(col)

In [16]:
drop_flags= ['FLAG_DOCUMENT_11_x', 'FLAG_DOCUMENT_14.1', 'FLAG_DOCUMENT_16_y',
'FLAG_DOCUMENT_18_y', 'FLAG_DOCUMENT_3_y', 'FLAG_DOCUMENT_4_y', 'FLAG_DOCUMENT_6_y', 
            'FLAG_OWN_CAR_y', 'FLAG_OWN_REALTY_y']


In [17]:
FLAG_DOCUMENT_7 = pd.to_numeric(master['FLAG_DOCUMENT_7_y'] + master['FLAG_DOCUMENT_7_x'])

In [18]:
FLAG_DOCUMENT_8 = pd.to_numeric(master['FLAG_DOCUMENT_8_y'] + master['FLAG_DOCUMENT_8_x'])

In [19]:
def prepare_flags(_df):
    _df['FLAG_DOCUMENT_7'] = pd.to_numeric(_df['FLAG_DOCUMENT_7_y'] + _df['FLAG_DOCUMENT_7_x'])
    _df['FLAG_DOCUMENT_8'] = pd.to_numeric(_df['FLAG_DOCUMENT_8_y'] + _df['FLAG_DOCUMENT_8_x'])

    # These are duplicate 
    dupes_flags= ['FLAG_DOCUMENT_11_x', 'FLAG_DOCUMENT_14.1', 'FLAG_DOCUMENT_16_y',
                'FLAG_DOCUMENT_18_y', 'FLAG_DOCUMENT_3_y', 'FLAG_DOCUMENT_4_y', 'FLAG_DOCUMENT_6_y', 
                'FLAG_OWN_CAR_y', 'FLAG_OWN_REALTY_y','FLAG_DOCUMENT_7_y','FLAG_DOCUMENT_7_x',
                'FLAG_DOCUMENT_8_y','FLAG_DOCUMENT_8_x']    
    
    _df = _df.drop(columns=dupes_flags)
    
    # Columns with a lot of Nans
    drop_cols=['FLAG_OWN_CAR_x','FLAG_DOCUMENT_16_x','FLAG_DOCUMENT_11_y'  ,'FLAG_DOCUMENT_2']
    
    ## Todo instead of dropping Nan maybe interpolate them?
    _df = _df.drop(columns=drop_cols)
    
    # Fixing dtypes
    _df['FLAG_OWN_REALTY_x'] = _df['FLAG_OWN_REALTY_x'].apply(lambda y: 1 if y =='Y' else 0)
    _df['flag_document_9'] = _df['flag_document_9'].astype(int)
    _df['flag_document_13'] = _df['flag_document_13'].astype(int)
    
    return _df

In [20]:
master_flags = prepare_flags(master)

In [21]:
flags = []
pattern = re.compile("FLAG|flag")
for col in master_flags.columns:
    if pattern.search(col):
        flags.append(col)

## X and Y

In [22]:
not_flags = list(set(master_flags.columns) - set(flags))
not_flags.sort()

In [23]:
def prepare_x_y_merge(_df):
    combine = ['AMT_CREDIT_x', 'AMT_INCOME_TOTAL_x', 'DEF_30_CNT_SOCIAL_CIRCLE_x', 
              'FLOORSMAX_AVG_x', 'LIVINGAPARTMENTS_AVG_x', 'OBS_60_CNT_SOCIAL_CIRCLE_x',
              'YEARS_BEGINEXPLUATATION_AVG_x' ]
    
    _df['AMT_CREDIT'] = pd.to_numeric(_df['AMT_CREDIT_x'] + _df['AMT_CREDIT_y'])
    _df['AMT_INCOME_TOTAL'] = pd.to_numeric(_df['AMT_INCOME_TOTAL_x'] + _df['AMT_INCOME_TOTAL_y'])
    _df['DEF_30_CNT_SOCIAL_CIRCLE'] = pd.to_numeric(_df['DEF_30_CNT_SOCIAL_CIRCLE_x'] + _df['DEF_30_CNT_SOCIAL_CIRCLE_y'])
    _df['FLOORSMAX_AVG'] = pd.to_numeric(_df['FLOORSMAX_AVG_x'] + _df['FLOORSMAX_AVG_y'])
    _df['LIVINGAPARTMENTS_AVG'] = pd.to_numeric(_df['LIVINGAPARTMENTS_AVG_x'] + _df['LIVINGAPARTMENTS_AVG_y'])
    _df['OBS_60_CNT_SOCIAL_CIRCLE'] = pd.to_numeric(_df['OBS_60_CNT_SOCIAL_CIRCLE_x'] + _df['OBS_60_CNT_SOCIAL_CIRCLE_y'])
    _df['YEARS_BEGINEXPLUATATION_AVG'] = pd.to_numeric(_df['YEARS_BEGINEXPLUATATION_AVG_x'] + _df['YEARS_BEGINEXPLUATATION_AVG_y'])

    _df = _df.drop(columns=combine)
    # Remove exact duplicates:
    duplicates = ['AMT_REQ_CREDIT_BUREAU_HOUR_y', 
    'CNT_FAM_MEMBERS_x', 'DAYS_EMPLOYED_y', 'NONLIVINGAPARTMENTS_AVG_y',
    'REGION_POPULATION_RELATIVE_y', 'REG_REGION_NOT_WORK_REGION_y',
    'WALLSMATERIAL_MODE_y', 'WEEKDAY_APPR_PROCESS_START_y']
    
    _df = _df.drop(columns=duplicates)
    
    # Low count
    low_count = ['AMT_REQ_CREDIT_BUREAU_DAY_y', 'APARTMENTS_MODE_y', 
                 'FLOORSMAX_MEDI_y', 'NAME_TYPE_SUITE_y', 'NONLIVINGAREA_MEDI_y']
    
    _df = _df.drop(columns=low_count)
    
    # Columns with a lot of Nans
    drop_cols=['AMT_REQ_CREDIT_BUREAU_DAY_x', 'APARTMENTS_MODE_x', 'FLOORSMAX_AVG', 'FLOORSMAX_MEDI_x',
              'LIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_MEDI_x']
    
    ## Todo instead of dropping Nan maybe interpolate them?
    _df = _df.drop(columns=drop_cols)
    
    return _df

In [24]:
master_dupes = prepare_x_y_merge(master_flags)

In [25]:
master_dupes.shape

(100000, 100)

In [26]:
master_dupes[['OCCUPATION_TYPE', 'OCCUPATION_TYPE.1']].head()

,OCCUPATION_TYPE,OCCUPATION_TYPE.1
SK_ID_CURR,,
100004,Laborers,Laborers
100006,Laborers,Laborers
100012,Laborers,Laborers
100014,Core staff,Core staff
100015,NaN,NaN


## Dot ones

In [27]:
def drop_dot_1(_df):
    dots = ['REGION_RATING_CLIENT_W_CITY.1', 'EXT_SOURCE_1.1', 'DAYS_REGISTRATION.1', 'EXT_SOURCE_3.1',
            'LIVINGAREA_MEDI.1', 'HOUR_APPR_PROCESS_START.1', 'FLOORSMIN_MEDI.1', 'OCCUPATION_TYPE.1']
    
    _df = _df.drop(columns=dots)
    
    return _df

In [28]:
md = drop_dot_1(master_dupes)

## Nulls

In [29]:
lots_of_nas = []
for col in md.columns:
    if md[col].isnull().sum() > 25000:
        lots_of_nas.append(col)

print(lots_of_nas)

['BASEMENTAREA_AVG', 'ENTRANCES_AVG', 'FONDKAPREMONT_MODE', 'OWN_CAR_AGE', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'LIVINGAPARTMENTS_MEDI', 'WEEKDAY_APPR_PROCESS_START_x', 'WALLSMATERIAL_MODE_x', 'AMT_REQ_CREDIT_BUREAU_HOUR_x', 'REGION_POPULATION_RELATIVE_x', 'NONLIVINGAPARTMENTS_AVG_x', 'DAYS_EMPLOYED_x', 'CNT_FAM_MEMBERS_y', 'LIVE_REGION_NOT_WORK_REGION', 'REG_REGION_NOT_LIVE_REGION', 'NAME_HOUSING_TYPE', 'NAME_CONTRACT_TYPE', 'LIVINGAPARTMENTS_AVG_y', 'EXT_SOURCE_1', 'LIVINGAREA_MEDI', 'FLOORSMIN_MEDI', 'OCCUPATION_TYPE', 'LANDAREA_MEDI', 'LANDAREA_MEDI.1', 'YEARS_BEGINEXPLUATATION_AVG']


In [30]:
md[lots_of_nas].describe()

,BASEMENTAREA_AVG,ENTRANCES_AVG,OWN_CAR_AGE,YEARS_BUILD_AVG,COMMONAREA_AVG,LIVINGAPARTMENTS_MEDI,AMT_REQ_CREDIT_BUREAU_HOUR_x,REGION_POPULATION_RELATIVE_x,NONLIVINGAPARTMENTS_AVG_x,DAYS_EMPLOYED_x,CNT_FAM_MEMBERS_y,LIVE_REGION_NOT_WORK_REGION,REG_REGION_NOT_LIVE_REGION,EXT_SOURCE_1,LIVINGAREA_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LANDAREA_MEDI.1,YEARS_BEGINEXPLUATATION_AVG
count,41555.000000,49896.000000,34063.000000,33646.000000,30158.000000,31716.000000,56690.000000,65536.000000,20095.000000,65536.000000,65535.000000,50000.000000,50000.000000,43778.000000,49968.000000,32277.000000,40750.000000,40750.000000,51348.000000
mean,0.088648,0.149930,12.029856,0.751972,0.045082,0.102284,0.005962,0.020864,0.009111,63824.941711,2.153796,0.040160,0.014860,0.502039,0.109003,0.230678,0.067152,0.067152,0.977706
std,0.083451,0.100033,11.870149,0.113833,0.079005,0.095316,0.081223,0.013855,0.048577,141292.529375,0.909280,0.196336,0.120994,0.210384,0.113506,0.162485,0.082190,0.082190,0.058696
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000533,0.000000,-17170.000000,1.000000,0.000000,0.000000,0.017177,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.044300,0.069000,5.000000,0.687200,0.007800,0.051300,0.000000,0.010006,0.000000,-2770.000000,2.000000,0.000000,0.000000,0.334178,0.045800,0.083300,0.018300,0.018300,0.976700
50%,0.076500,0.137900,9.000000,0.755200,0.020900,0.076100,0.000000,0.018850,0.000000,-1225.000000,2.000000,0.000000,0.000000,0.504573,0.074800,0.208300,0.048700,0.048700,0.981600
75%,0.112000,0.206900,15.000000,0.823200,0.051300,0.123100,0.000000,0.028663,0.003900,-291.000000,3.000000,0.000000,0.000000,0.675197,0.131000,0.375000,0.086800,0.086800,0.986600
max,1.000000,1.000000,91.000000,1.000000,1.000000,1.000000,3.000000,0.072508,1.000000,365243.000000,13.000000,1.000000,1.000000,0.946098,1.000000,1.000000,1.000000,1.000000,1.000000


In [31]:
def drop_nans(_df):
    lots_of_nans = ['IS_KAEL', 'BASEMENTAREA_AVG', 'ENTRANCES_AVG', 'FONDKAPREMONT_MODE', 
                    'OWN_CAR_AGE', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 
                    'LIVINGAPARTMENTS_MEDI', 'WEEKDAY_APPR_PROCESS_START_x', 
                    'WALLSMATERIAL_MODE_x', 'AMT_REQ_CREDIT_BUREAU_HOUR_x', 
                    'REGION_POPULATION_RELATIVE_x', 'NONLIVINGAPARTMENTS_AVG_x', 
                    'DAYS_EMPLOYED_x', 'CNT_FAM_MEMBERS_y', 'LIVE_REGION_NOT_WORK_REGION', 
                    'REG_REGION_NOT_LIVE_REGION', 'NAME_HOUSING_TYPE', 'NAME_CONTRACT_TYPE', 
                    'LIVINGAPARTMENTS_AVG_y', 'EXT_SOURCE_1', 'LIVINGAREA_MEDI', 'FLOORSMIN_MEDI',
                    'OCCUPATION_TYPE', 'LANDAREA_MEDI', 'LANDAREA_MEDI.1', 'YEARS_BEGINEXPLUATATION_AVG',
                    # Empty?
                    'DEF_30_CNT_SOCIAL_CIRCLE_y', 'OBS_60_CNT_SOCIAL_CIRCLE_y', 'REG_REGION_NOT_WORK_REGION_x',
                   'AMT_CREDIT_y', 'AMT_INCOME_TOTAL_y']
    
    _df = _df.drop(columns=lots_of_nans)
    
    # These have not so many nans but a lot for the target variable
    not_so_many= ['EXT_SOURCE_3','YEARS_BEGINEXPLUATATION_AVG_y','FLOORSMAX_AVG_y']
    _df = _df.drop(columns=not_so_many)
    
    # We are using the average for the others
    means = ['DEF_60_CNT_SOCIAL_CIRCLE',
     'AMT_ANNUITY', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE']
    
    for col in means:
        _df[col] = _df[col].fillna(_df[col].mean())
        
    return _df

In [32]:
mf = drop_nans(md)

## Dtypes

In [33]:
cats = ['NAME_TYPE_SUITE_x', 'CODE_GENDER', 'NAME_FAMILY_STATUS', 'name_education_type', 'name_income_type']

In [34]:
to_numeric = ['AMT_GOODS_PRICE', 'EXT_SOURCE_2', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT']

In [35]:
wide_cats = ['ORGANIZATION_TYPE', 'ORGANIZATION_TYPE_CAT']

In [36]:
def prepare_dtypes(_df):
    interrogation  = ['FLOORSMIN_AVG', 'TOTALAREA_MODE', 'FLOORSMIN_MODE', 'NONLIVINGAREA_AVG', 'HOUSETYPE_MODE', 'LANDAREA_AVG',
    'ELEVATORS_AVG', 'COMMONAREA_MODE']
    
    _df = _df.drop(columns=interrogation)
    
    # categoricals to dummies
    cats = ['NAME_TYPE_SUITE_x', 'CODE_GENDER', 'NAME_FAMILY_STATUS', 'name_education_type', 'name_income_type']
    
    _df = pd.get_dummies(_df, columns=cats, drop_first=True)
    
    #convert
    to_numeric = ['AMT_GOODS_PRICE', 'EXT_SOURCE_2', 'REGION_RATING_CLIENT']
    
    _df['AMT_GOODS_PRICE'] = pd.to_numeric(_df['AMT_GOODS_PRICE'].apply(lambda x: x.split('€')[0]))
    _df['EXT_SOURCE_2'] = pd.to_numeric(_df['EXT_SOURCE_2'].apply(lambda x: x.split('%')[0]), errors='coerce')
    _df.loc[100004,'REGION_RATING_CLIENT'] = 2
    
    #drop wide cats
    wide_cats = ['ORGANIZATION_TYPE', 'ORGANIZATION_TYPE_CAT']
    _df = _df.drop(columns=wide_cats)
    
    _df['EXT_SOURCE_2'] = _df['EXT_SOURCE_2'].fillna(_df['EXT_SOURCE_2'].mean())
    
    return _df

In [37]:
m_dtypes = prepare_dtypes(mf)

In [38]:
m_dtypes.isnull().sum().sort_values()

days_birth                                               0
flag_document_9                                          0
obs_30_cnt_social_circle                                 0
FLAG_DOCUMENT_7                                          0
FLAG_DOCUMENT_8                                          0
AMT_CREDIT                                               0
AMT_INCOME_TOTAL                                         0
DEF_30_CNT_SOCIAL_CIRCLE                                 0
OBS_60_CNT_SOCIAL_CIRCLE                                 0
NAME_TYPE_SUITE_x_Children                               0
NAME_TYPE_SUITE_x_Family                                 0
NAME_TYPE_SUITE_x_Group of people                        0
NAME_TYPE_SUITE_x_Other_B                                0
NAME_TYPE_SUITE_x_Spouse, partner                        0
NAME_TYPE_SUITE_x_Unaccompanied                          0
NAME_TYPE_SUITE_x_�Q��                                   0
CODE_GENDER_M                                           

In [ ]:
m_dtypes.to_csv('../data/processed/master_2.csv')